In [1]:
from pyspark import SparkConf, SparkContext
# 스파크 환경 설정 객체 생성
conf = SparkConf().setMaster('local').setAppName('restaurant-review-average')
spark = SparkContext(conf=conf).getOrCreate()

24/12/05 10:16:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
import os
directory = os.path.join(os.getcwd(), 'data')
filename = 'restaurant_reviews.csv'
filepath = os.path.join(directory, filename)

In [7]:
# RDD 생성
lines = spark.textFile('file:///'+filepath.replace('\\', '/'))
lines.take(5)

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,']

In [9]:
header = lines.first()
filtered_lines = lines.filter(lambda row : row != header)
filtered_lines.take(5)

['0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,']

In [10]:
def parse(row):
    fields = row.split(',')
    category = fields[2]
    review = int(fields[3])
    
    return category, review

In [12]:
result = filtered_lines.map(parse)
result.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [13]:
cat_review = filtered_lines.map(parse).persist()
cat_review

PythonRDD[11] at RDD at PythonRDD.scala:53

In [14]:
result1 = cat_review.take(10)
result2 = cat_review.collect()
result1

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [15]:
result2

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [16]:
spark.stop()